# 뉴스그룹 데이터 대한 이해

**1) 20개의 다른 주제를 가진 뉴스그룹 데이터의 훈련 데이터를 다운로드**

**2) 데이터가 어떤 속성으로 구성되어있는지 출력**

In [3]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


**3) 훈련용 샘플의 개수를 확인<br>**
&nbsp;&nbsp;&nbsp;&nbsp;     **뉴스그룹 데이터는 이미 훈련 데이터와 테스트 데이터를 미리 분류해두었음을 알 수 있음**

In [4]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


**4) 카테고리 구성 출력**

In [5]:
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


**5) 첫 번째 샘플의 카테고리 확인**

In [6]:
print(newsdata.target[0])

7


In [7]:
print(newsdata.target_names[7])

rec.autos


**6) 첫 번재 샘플의 내용 확인**<br>
&nbsp;&nbsp;&nbsp;&nbsp;      **스포츠 카에 대한 글이 0부터 19까지의 카테고리 중 7번 레이블에 속하는 글임을 알 수 있음**

In [8]:
print(newsdata.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







# 나이브 베이즈 분류

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score # 정확도 계산

**1) 나이브 베이즈 분류를 사용하기 위해 다운로드 받은 훈련 데이터를 BoW로 만들어 전처리**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **11314개의 훈련용 샘플의 개수(문서의 수)와 130107개의 전체 훈련 데이터에 등장한 단어의 수를 가진 DTM이 만들어짐**

In [10]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


**2) TF-IDF 행렬을 생성**<br>
&nbsp;&nbsp;&nbsp;&nbsp;      **DTM을 그래도 나이브 베이즈 분류기에 사용하는 것보다는**<br>
&nbsp;&nbsp;&nbsp;&nbsp;      **TF-IDF 가중치를 적용한 TF-IDF 행렬을 입력으로 텍스트 분류를 수행하면 성능의 개선을 얻을 수 있음**

In [11]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


**3) 나이브 베이즈 모델을 사용해 라플라스 스무딩을 적용하여 나이브 베이즈 분류를 수행**

In [12]:
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB()

In [13]:
# alpha=1.0은 라플라스 스무딩 적용
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

**4) 테스트 데이터를 통한 예측값과 실제값에 대한 정확도 확인**

In [14]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) # 테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) # 테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) # DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) # 테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) # 예측값과 실제값 비교

정확도: 0.7738980350504514
